In [10]:
import pandas as pd
# Performance Evaluation tools
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import TimeSeriesSplit



df_train = pd.read_csv('./dataAfterPruned/tree2_train.csv')
df_test = pd.read_csv('./dataAfterPruned/tree2_test.csv')


In [2]:
from imblearn.under_sampling import NearMiss


X = df_train.drop('fraud_ind', axis=1)
y = df_train['fraud_ind']
# Since this data set is imbalance, use under sampling, for better training result of models 
X_nearmiss, y_nearmiss = NearMiss(n_jobs=-1).fit_resample(X, y)

print(X_nearmiss.shape)
print(y_nearmiss.shape)
print(X.shape)
print(y.shape)

(30566, 23)
(30566,)
(1141340, 23)
(1141340,)


In [4]:
from tpot import TPOTClassifier

pipeline_optimizer = TPOTClassifier(generations=10, population_size=200, cv=5,
                                    random_state=42, verbosity=2, early_stop= 12,
                                    scoring = 'f1',  n_jobs=-2)

pipeline_optimizer.fit(X_nearmiss, y_nearmiss)
# pipeline_optimizer.score(X_test, y_test)


# TPOT will evaluate population_size + generations × offspring_size pipelines in total.
# 100 + 8*100 = 900 for 3 hours
# 200 + 10*100 = 1200 for ?



Generation 1 - Current best internal CV score: 0.9062494560845027

Generation 2 - Current best internal CV score: 0.9062494560845027

Generation 3 - Current best internal CV score: 0.9062494560845027

Generation 4 - Current best internal CV score: 0.9062494560845027

Generation 5 - Current best internal CV score: 0.9062494560845027

Generation 6 - Current best internal CV score: 0.9062494560845027

Generation 7 - Current best internal CV score: 0.9063186862113612

Generation 8 - Current best internal CV score: 0.9063186862113612

Generation 9 - Current best internal CV score: 0.9070076889960192

Generation 10 - Current best internal CV score: 0.9070076889960192

Best pipeline: GradientBoostingClassifier(MaxAbsScaler(input_matrix), learning_rate=0.5, max_depth=10, max_features=1.0, min_samples_leaf=13, min_samples_split=4, n_estimators=100, subsample=1.0)


NameError: name 'X_test' is not defined

In [5]:
pipeline_optimizer.export('./tpot_exported_pipeline2.py')


In [6]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(learning_rate=0.5, max_depth=10, max_features=1.0, min_samples_leaf=13, min_samples_split=4, n_estimators=100, subsample=1.0)



In [7]:
def basicTrainAndTest(train, test, model, name):
    X = train.drop('fraud_ind', axis=1).values
    y = train['fraud_ind'].values
    
    # For prediction
    
    X_test = test.drop('fraud_ind', axis=1).values
    y_test = test['fraud_ind'].values
    
    model.fit(X, y)
    y_pred = model.predict(X_test)
    
    print(f'Dataset: {name}')
    print('---' * 45)
    print(f'Accuracy: {model.score(X_test, y_test)}')
    print(f'Precision: {precision_score(y_test, y_pred)}')
    print(f'Reacall: {recall_score(y_test, y_pred)}')
    print(f'f1-score: {f1_score(y_test, y_pred)}')
    print(f"micro f1-score: {f1_score(y_test, y_pred, average='micro')}")
    print(f"macro f1-score: {f1_score(y_test, y_pred, average='macro')}")
    print(f"weighted f1-score: {f1_score(y_test, y_pred, average='weighted')}")
    print(f'AUC: {roc_auc_score(y_test, y_pred)}')
    print('---' * 45)
    

In [11]:

basicTrainAndTest(df_train, df_test, clf, "GradientBoostingClassifier")

# Dataset: GradientBoostingClassifier
# ---------------------------------------------------------------------------------------------------------------------------------------
# Accuracy: 0.989309943303535
# Precision: 0.6066199872692553
# Reacall: 0.5636829652996845
# f1-score: 0.5843638221768013
# micro f1-score: 0.989309943303535
# macro f1-score: 0.7894745809938337
# weighted f1-score: 0.9891163952865567
# AUC: 0.7793719521803119
# ---------------------------


Dataset: GradientBoostingClassifier
---------------------------------------------------------------------------------------------------------------------------------------
Accuracy: 0.989309943303535
Precision: 0.6066199872692553
Reacall: 0.5636829652996845
f1-score: 0.5843638221768013
micro f1-score: 0.989309943303535
macro f1-score: 0.7894745809938337
weighted f1-score: 0.9891163952865567
AUC: 0.7793719521803119
---------------------------------------------------------------------------------------------------------------------------------------
